In [2]:
# coding: utf-8

# In[ ]:


import requests             
from bs4 import BeautifulSoup 
import csv                  
import webbrowser
import io

def display(content, filename='output.html'):
    with open(filename, 'wb') as f:
        f.write(content)
        webbrowser.open(filename)

def get_soup(session, url, show=False):
    r = session.get(url)
    if show:
        display(r.content, 'temp.html')

    if r.status_code != 200: # not OK
        print('[get_soup] status code:', r.status_code)
    else:
        return BeautifulSoup(r.text, 'html.parser')
    
def post_soup(session, url, params, show=False):
    '''Read HTML from server and convert to Soup'''

    r = session.post(url, data=params)
    
    if show:
        display(r.content, 'temp.html')

    if r.status_code != 200: # not OK
        print('[post_soup] status code:', r.status_code)
    else:
        return BeautifulSoup(r.text, 'html.parser')
    
def scrape(url, lang='ALL'):

    # create session to keep all cookies (etc.) between requests
    session = requests.Session()

    session.headers.update({
        'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:57.0) Gecko/20100101 Firefox/57.0',
    })


    items = parse(session, url + '?filterLang=' + lang)

    return items

def parse(session, url):
    '''Get number of reviews and start getting subpages with reviews'''

    print('[parse] url:', url)

    soup = get_soup(session, url)

    if not soup:
        print('[parse] no soup:', url)
        return

    #num_reviews = soup.find('span', class_='reviews_header_count').text # get text
    num_reviews = soup.find('span', class_='reviews_header_count')
    num_reviews = num_reviews.text
    num_reviews = num_reviews[1:-1] 
    num_reviews = num_reviews.replace(',', '')
    num_reviews = int(num_reviews) # convert text into integer
    print('[parse] num_reviews ALL:', num_reviews)

    url_template = url.replace('.html', '-or{}.html')
    print('[parse] url_template:', url_template)

    items = []

    offset = 0

    while(True):
        subpage_url = url_template.format(offset)

        subpage_items = parse_reviews(session, subpage_url)
        if not subpage_items:
            break

        items += subpage_items

        if len(subpage_items) < 5:
            break

        offset += 5

    return items

def get_reviews_ids(soup):

    items = soup.find_all('div', attrs={'data-reviewid': True})

    if items:
        reviews_ids = [x.attrs['data-reviewid'] for x in items][::2]
        print('[get_reviews_ids] data-reviewid:', reviews_ids)
        return reviews_ids
    
def get_more(session, reviews_ids):

    url = 'https://www.tripadvisor.com/OverlayWidgetAjax?Mode=EXPANDED_HOTEL_REVIEWS_RESP&metaReferer=Hotel_Review'

    payload = {
        'reviews': ','.join(reviews_ids), # ie. "577882734,577547902,577300887",
        #'contextChoice': 'DETAIL_HR', # ???
        'widgetChoice': 'EXPANDED_HOTEL_REVIEW_HSX', # ???
        'haveJses': 'earlyRequireDefine,amdearly,global_error,long_lived_global,apg-Hotel_Review,apg-Hotel_Review-in,bootstrap,desktop-rooms-guests-dust-en_US,responsive-calendar-templates-dust-en_US,taevents',
        'haveCsses': 'apg-Hotel_Review-in',
        'Action': 'install',
    }

    soup = post_soup(session, url, payload)

    return soup

def parse_reviews(session, url):
    '''Get all reviews from one page'''

    print('[parse_reviews] url:', url)

    soup =  get_soup(session, url)

    if not soup:
        print('[parse_reviews] no soup:', url)
        return

    hotel_name = soup.find('h1', id='HEADING').text

    reviews_ids = get_reviews_ids(soup)
    if not reviews_ids:
        return

    soup = get_more(session, reviews_ids)

    if not soup:
        print('[parse_reviews] no soup:', url)
        return

    items = []

    for idx, review in enumerate(soup.find_all('div', class_='reviewSelector')):

        badgets = review.find_all('span', class_='badgetext')
        if len(badgets) > 0:
            contributions = badgets[0].text
        else:
            contributions = '0'

        if len(badgets) > 1:
            helpful_vote = badgets[1].text
        else:
            helpful_vote = '0'
        user_loc = review.select_one('div.userLoc strong')
        if user_loc:
            user_loc = user_loc.text
        else:
            user_loc = ''
            
        bubble_rating = review.select_one('span.ui_bubble_rating')['class']
        bubble_rating = bubble_rating[1].split('_')[-1]
        #print (bubble_rating)
        item = {
            'review_body': review.find('p', class_='partial_entry').text,
            #'review_date': review.find('span', class_='ratingDate')['title'], # 'ratingDate' instead of 'relativeDate'
            'review_rating' : bubble_rating
        }

        items.append(item)
        print('\n--- review ---\n')
        for key,val in item.items():
            print(' ', key, ':', val)

    print()

    return items

def write_in_csv(items, filename='results.csv',
                  headers=['hotel name', 'review title', 'review body',
                           'review date', 'contributions', 'helpful vote',
                           'user name' , 'user location', 'rating'],
                  mode='w'):

    print('--- CSV ---')

    with io.open(filename, mode, encoding="utf-8") as csvfile:
        csv_file = csv.DictWriter(csvfile, headers)

        if mode == 'w':
            csv_file.writeheader()

        csv_file.writerows(items)
        
DB_COLUMN   = 'review_body'
DB_COLUMN1 = 'review_date'
DB_COLUMN2 = 'review_rating'

start_urls = [
    'https://www.tripadvisor.com/Attraction_Review-g297701-d1515658-Reviews-Tegalalang_Rice_Terrace-Ubud_Gianyar_Bali.html',
    'https://www.tripadvisor.com/Attraction_Review-g297699-d6394471-Reviews-Yeh_Sanih_Spring-Singaraja_Bali.html',
    'https://www.tripadvisor.com/Attraction_Review-g2283829-d379340-Reviews-Tanah_Lot_Temple-Beraban_Bali.html',
]

lang = 'id'

headers = [ 
    DB_COLUMN, 
    #DB_COLUMN1, 
    DB_COLUMN2,
]

for url in start_urls:

    # get all reviews for 'url' and 'lang'
    items = scrape(url, lang)

    if not items:
        print('No reviews')
    else:
        # write in CSV
        print("else")
        filename = url.split('Reviews-')[1][:-5] + '__' + lang
        print('filename:', filename)
        write_in_csv(items, filename + '.csv', headers, mode='w')
        
#span class="ui_bubble_rating bubble_30" buat cari rating


[parse] url: https://www.tripadvisor.com/Attraction_Review-g297701-d1515658-Reviews-Tegalalang_Rice_Terrace-Ubud_Gianyar_Bali.html?filterLang=id
[parse] num_reviews ALL: 14053
[parse] url_template: https://www.tripadvisor.com/Attraction_Review-g297701-d1515658-Reviews-Tegalalang_Rice_Terrace-Ubud_Gianyar_Bali-or{}.html?filterLang=id
[parse_reviews] url: https://www.tripadvisor.com/Attraction_Review-g297701-d1515658-Reviews-Tegalalang_Rice_Terrace-Ubud_Gianyar_Bali-or0.html?filterLang=id
[get_reviews_ids] data-reviewid: ['642530185', '638767490', '636845556', '633371086', '632543555', '632106330', '631812013', '631448813', '627627315', '627102465']

--- review ---

  review_body : Wisata ke bali tidak hanya mengunjungi pantai, tapi juga bisa menikmati indahnya pemandangan. Tegalalang termasuk destinasi favorit keluarga, karna disini kita bisa merasakan segar nya udara dg pemandangan hijau nya sawah yg indah. 
  review_rating : 50

--- review ---

  review_body : Tempat yang cukup sejuk 

[get_reviews_ids] data-reviewid: ['623546515', '623057811', '620680930', '619072882', '614990748', '614729981', '614232735', '614123172', '611189128', '610360747']

--- review ---

  review_body : Pemandangannya sangat bagus dan indah tapi sayang ,banyak orang dan tidak kelihatan natural lagi dan macetnya luar biasa 
  review_rating : 30

--- review ---

  review_body : Jika ingin turun berjalan ke bawah sawah harus melewati cafe-cafe di pinggir jalan.Jalan untuk turun ke bawah sangat sempit harus hati-hati.Disini terdapat toko kecil yang menjual gelato dan jus...perjalanan yang melegakan.
  review_rating : 50

--- review ---

  review_body :  Bersyukur banget saya lahir dibali,, banyak memiliki tempat wisata yg sangat bagus. Bagus. Pemandangan sawahnya yg hijauspt ini membuat saya tidak ingin meninggalkan tempat ini.I love Bali
  review_rating : 40

--- review ---

  review_body : Untuk masuk ke tegalalang ini biasanya tidak dipungut biaya, akan tetapi di dekat pintu masuknya biasanya

[get_reviews_ids] data-reviewid: ['587830668', '587289117', '586889965', '585089239', '585053321', '585001518', '583530854', '582811397', '582638048', '582440356']

--- review ---

  review_body : Saya beberapa kali berkunjung kemari, tetapi tidak pernah bosan. Tiap ke Ubud pasti saya sempatkan main kemari
  review_rating : 50

--- review ---

  review_body : Tanaman padi dengan sistem subak di kawasan tegal lalang sangat terkenal dengan keindahannya. banyak wisatawan yang berhenti untuk mengambil gambar, dengan keindahan pemandangan terras,namun sayangnya banyak pedagang acung yang sedikit menggangu untuk menawarkan dagangannya.
  review_rating : 50

--- review ---

  review_body : View sawah yg bgus sekalli denn pemandangan sawah yg sangat indah sambil menikmati makanan dan minuman sambil.bersantaiDan lbh baik dteng pg to sore nya biar lbh sejuh
  review_rating : 40

--- review ---

  review_body : Kunjungan ke teras sawah tegagalang sangat recommended karena view yang diberikan sang

[get_reviews_ids] data-reviewid: ['582110912', '577875977', '574292515', '571195563', '570401563', '567609641', '566607091', '565426369', '565287756', '562298822']

--- review ---

  review_body : Pemandangan yang memanjakan mata. Sawah bertingkat-tingkat yang luar biasa, ada juga ayunan disekitarnya yang lumayan memacu adrenalin.
  review_rating : 40

--- review ---

  review_body : Berkunjunglah di sore hari...Lebih asik.. Sembari nikmati kopi..Bisa duduk2 di kafe, atau naik turun ke swah lihat indahnya terasering/sengkedan buatan para petani yang diakui oleh dunia sebagai world heritage. 
  review_rating : 50

--- review ---

  review_body : Jika bosan dengan suasana pantai, maka bergeraklah ke arah Ubud. Disini kita bisa dimanjakan dengan hijaunya hamparan sawah yang jarang ditemukan di perkotaan 
  review_rating : 50

--- review ---

  review_body : Apabila ke Tegalalang saya rekomendasikan agar jangan lupa mencoba dan mengunjungi tempat yang satu ini bersama keluarga maupun pasan

[get_reviews_ids] data-reviewid: ['542645173', '542526486', '542352156', '541686850', '541412805', '538389123', '537780751', '537771201', '537759604', '533574177']

--- review ---

  review_body : Saya tidak pernah bosan untuk pergi ke sana, selain saya tour guide.. juga saya sangat menikmati pemandangan nya. sangat adventure
  review_rating : 50

--- review ---

  review_body : Tempat ini menarik untuk pelancong yang suka bergambar. Pemandangan sawah berlereng, tapi ada di beberapa tempat yang dikenakan tol oleh penduduk.
  review_rating : 30

--- review ---

  review_body : Tetap terpukau tiap berkunjung kesini. Kali ini pengunjung bisa megakses dan jalan - jalan hingga ke tengah sawah. dan juga sudah ada warung - warung kopi dan tempat istirahat di area persawahan. 
  review_rating : 40

--- review ---

  review_body : teras sawah tegallalang adl suatu tempat wisata yg sangat menarik & menyegarkan hati, selain memiliki pemandangan hijau yg amat luarbiasa juga keunikan dari bentuk pe

...

[get_reviews_ids] data-reviewid: ['198136982', '197802069', '197793921', '197792508', '197046885', '196609674', '196491816', '196373703', '190100727', '187932448']

--- review ---

  review_body : Jika anda ke tanah lot, saya sarankan untuk berada disana sebelum matahari tenggelam. Sunset di tanah lot merupakan pemandangan indah yang takkan terlupakan. selain memiliki sunset yang menawan tanah lot juga bisa menjadi latar belakang untuk berfoto, terutama bagi pasangan yang sedang dilanda asmara tempat ini cocok untuk menjadi latar belakang foto nan romantis. selain itu banyaknya pernak-pernik dengan harga yang cukup murah membuat tempat ini sangat menyenangkan sehingga kita dapat membeli oleh-oleh khas bali untuk orang-orang terkasih tanpa harus berkunjung ke tempat yang lainnya.
  review_rating : 40

--- review ---

  review_body : Pura Tanah Lot merupakan tempat favorit saya di Bali, tempat disini sangat indah, terutama untuk photoshoot, sangat sayang kalau ngga kesini, wajib dikunjun

[get_reviews_ids] data-reviewid: ['187521339', '184550648', '184360530', '183002318', '180415876', '179008484', '176398217', '176121173', '175833306', '175660922']

--- review ---

  review_body : Sungguh indah menyaksikan sunset di tanah Lot Bali, diiringi deburan ombak sangat romantis sekali. Saya sangat senang dengan pemandangan dan pelayanan guide di Bali.
  review_rating : 40

--- review ---

  review_body : Ini kunjungan pertama saya, sebelum menuju ke sini, saya dan suami berkunjung ke Uluwatau, karean lebih dekat dari hotel tempat kami menginap (di Kawasan Nusa Dua). Jarak yang ditempuh dari UIluwatu ke Tanah Lot sekitar 2 jam. Tempat ini lebih enak dikunjungi ketika menjalang sore. Selain karena cuaca yang tak terlalu panas, tempat ini memiliki pemandangan yang bagus saat sunset.
  review_rating : 50

--- review ---

  review_body : Saya sudah sering kesini, tp tidak mebosankan. Tempat ini wajib Anda kunjungi jika berlibur ke bali. Anda akan dibawa ke dunia fantasi, amazing...

[get_reviews_ids] data-reviewid: ['168498153', '165834123', '162961707', '161877862', '160522524', '159495443', '158580982', '157995131', '156449716', '155420277']

--- review ---

  review_body : Dengan pemandangan pantainya dan deburan ombak yang membuat pura yang berdiri tegak di atas bukit karang ini menjadi semakin menarik untuk dikunjungi. Bila kita sedang beruntung maka pemandangan pura dan matahari tenggelam adalah sesuatu yang sangat mempesona di tempat ini. Selain itu ada pula ular yang dianggap keramat berada di goa yang berada di bagian bawah bukit karang tersebut. Kita dapat merencanakan perjalanan ke Pura Tanah Lot ini sesudah terlebih dahulu mengunjungi Mengwi atau Pura Taman Ayun.
  review_rating : 40

--- review ---

  review_body : Bila anda menyukai keindahan budaya, laut dan landscape maka destinasi ini wajib di kunjungi. Waktu terbaik untuk berkunjung adalah sewaktu air laut surut sehingga anda bisa melihat dan melintas di batu2 cadas.
  review_rating : 40

--- rev

[get_reviews_ids] data-reviewid: ['155292528', '155130683', '154055333', '153567556', '151130586', '149244372', '148496956', '147357270', '146048745', '145878877']

--- review ---

  review_body : Pantai Tanah Lot merupakan salah satu pantai yang ada di Pulau Bali yang perlu kita kunjungi karna disana terdapat banyak keindahan2 yang dapat memanjakan mata kita terutama pura yang ada di tengah laut klau air laut pasang tapi klau tidak kita bisa berjalan kaki saja untuk menuju pura tersebut disana ada mata air yang suci dan ular suci yang dipercaya orang sana sangat keramat. Disana kita juga bisa menikmati makanan khas Bali dan Pusat Oleh2 yang sangat terjangkau. So klau ke Bali jangan lupa ya mampir ke Pantai satu ini karna tempat nya sangat indah sekali....
  review_rating : 40

--- review ---

  review_body : saat itu, saya bersama teman-teman sekolah tinggi pariwisata Trisakti Jakarta berkunjung ke Bali, Denpasar, Indonesia :) . saya sangat senang dapat pergi kesana karena itu adalah 


--- review ---

  review_body : lokasi pura yang mengagumkan yang dikagumi oleh touris lokal maupun asing,tidak hanya pura tanah lot tapi juga keindahan alam di sekitar pura
  review_rating : 50

--- review ---

  review_body : Deru ombak yg berlomba menepi, memecah kesunyian. Pura Tanah Lot begitu menawan, apalagi saat air laut pasang, seakan pura yang berada diatas bebatuan mengapung di lautan. Sepanjang perjalanan ke Pura Tanah Lot dari area parkir kita sudah disapa banyak toko yg menjual souvenir khas Bali. Ditambah taman yang indah dan hijau sangat bagus dan eksotis bagi mereka penggemar photography.
  review_rating : 50

--- review ---

  review_body : Pemandangan yg indah. Terakhir kali kami ke sana, kami menemukan banyak kepiting berwarna ungu di sela2 karang, menarik sekali. Yang selalu kurang dari tempat wisata di Indonesia adalah toilet yang bersih.
  review_rating : 50

--- review ---

  review_body : Hebat Denpasar punya tempat se-eksotis Tanah Lot, Mungkin bnyak tempat i